In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import Compose, PILToTensor, Resize, Normalize
from PIL import Image
import os
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

True
1
0
NVIDIA GeForce RTX 3050 6GB Laptop GPU
Using device: cuda

NVIDIA GeForce RTX 3050 6GB Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
from torchvision.transforms import Compose, ToTensor, Resize, Normalize
cloudy = './Satellite/cloudy'
desert = './Satellite/desert'
green = './Satellite/green_area'
water = './Satellite/water'

transform = Compose([
    Resize((256, 256)),
    ToTensor(),
    Normalize(mean=(0.5,), std=(0.5,))
])

In [4]:
class Satellite(Dataset):
    def __init__(self, folder, label, transform=None):
        self.folder = folder
        self.label = label
        self.transform = transform
        self.images = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith('.jpg') or file.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, self.label
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            return None

# dataset
cloudy = Satellite(cloudy, label=0, transform=transform)
desert = Satellite(desert, label=1, transform=transform)
green = Satellite(green, label=2, transform=transform)
water = Satellite(water, label=3, transform=transform)

# dataloader
full_dataset = cloudy + desert+ green+ water
train_loader = DataLoader(full_dataset, batch_size=32, shuffle=True)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, input_height=256, input_width=256):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        self.fc1_input_size = self._get_flattened_size(input_height, input_width)
        self.fc1 = nn.Linear(self.fc1_input_size, 1024)
        self.fc2 = nn.Linear(1024, 4)

    def _get_flattened_size(self, height, width):
        dummy_input = torch.zeros(1, 3, height, width)  # Batch size = 1
        x = F.relu(self.bn1(self.conv1(dummy_input)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)
        return x.numel() 

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)  
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)  
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)  
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)  
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)  

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

In [7]:
criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        
        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")


Epoch 1, Loss: 1.4930134424939752
Epoch 2, Loss: 0.4216503955593163
Epoch 3, Loss: 0.2869491795480082
Epoch 4, Loss: 0.2428474008232694
Epoch 5, Loss: 0.17280361543155529
Epoch 6, Loss: 0.15955108124762774
Epoch 7, Loss: 0.17518840558477677
Epoch 8, Loss: 0.1413421927189285
Epoch 9, Loss: 0.09944582909850976
Epoch 10, Loss: 0.1135892330732366


In [9]:
def calculate_accuracy(model, data_loader, device):
    model.eval()  
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in data_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

train_accuracy = calculate_accuracy(model, train_loader, device)
print(f"Training Accuracy: {train_accuracy:.2f}%")

Training Accuracy: 97.80%
